# try to parallelize reading in csvs using dask?


In [ ]:
import dask
from dask import delayed
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster, progress
import re
import os
from scripts import get_tripdata
import zipfile

In [ ]:
try:
    client
except NameError:
    print("client does not exist!")
    # create LocalCluster w/ dask
    cluster = LocalCluster()
    client = Client(cluster)
    # then look at diagnostic dash here http://localhost:8787/status or http://127.0.0.1:8787/status

In [ ]:
cluster

In [ ]:
client
# client.shutdown()
# client.close()
# cluster.close()

In [ ]:
def unzip(item, zip_dir):
    """Unzips files
    Returns:
        Nothing
    """
    extension = ".zip"
    file_name = delayed()
    if item.endswith(extension):
        # create zipfile object and extract
        file_name = zip_dir + item
        with zipfile.ZipFile(file_name, "r") as zip_ref:
            zip_ref.extractall(zip_dir)
            unzipped = (zip_ref.namelist()[0]).replace(" ", "").lower()
            os.rename(f'{zip_dir}{zip_ref.namelist()[0]}', f'{zip_dir}{unzipped}')
        return unzipped

def get_list_of_files(x):
    return x

def clean_filenames(unzipped, zip_dir):
    if unzipped.endswith(".csv"):
        # check if name is formatted as YYYYMM-citibike-tripdata.csv
        if zip_dir == "../data/citibike-tripdata-jc/":
            result = re.match(r"jc-[0-9]{6}-citibike-tripdata.csv", unzipped)
            if not result: 
                print(f"{unzipped} does not match, changing name")
                yearmonth = re.findall(r'\d+', unzipped)[0]
                new_item_name = "jc-" + yearmonth + "-citibike-tripdata.csv"
                os.rename(f"{zip_dir}{unzipped}", f"{zip_dir}{new_item_name}")
        else: 
            result = re.match(r"[0-9]{6}-citibike-tripdata.csv", unzipped)
            if not result: 
                print(f"{unzipped} does not match, changing name")
                yearmonth = re.findall(r'\d+', unzipped)
                yearmonth = "".join(str(i) for i in re.findall(r'\d+', unzipped))
                new_item_name = yearmonth + "-citibike-tripdata.csv"
                os.rename(f"{zip_dir}{unzipped}", f"{zip_dir}{new_item_name}")

## attempt 1

In [ ]:
## this syntax is wrong for some reason. also slow
%%time
filelist = os.listdir(zip_dir)
zip_dir = "../data/citibike-tripdata-nyc/"
for item in filelist:
    lazy_unzip = unzip(item, zip_dir)

In [ ]:
%time 
dask.compute(*lazy_unzip)

In [ ]:
lazy_unzip.visualize()

## attempt 1a

In [ ]:
## run it
%%time
filelist = os.listdir(zip_dir)
zip_dir = "../data/citibike-tripdata-nyc/"
for item in filelist:
    lazy_unzip = unzip(item, zip_dir).compute()

## attempt 2

In [ ]:
# note: this doesn't work
# note: client.map submits many function calls


## NYC
# get_file_list = client.map(os.listdir, "data/citibike-tripdata-nyc/")
unzipped_nyc_files = client.map(unzip, [os.listdir("../data/citibike-tripdata-nyc/")], ["../data/citibike-tripdata-nyc/"])
clean_nyc = client.map(clean_filenames, [os.listdir("../data/citibike-tripdata-nyc/")], ["../data/citibike-tripdata-nyc/"])

## Jersey City
# unzipped_jc_files = client.map(unzip, os.listdir("data/citibike-tripdata-jc/"), "data/citibike-tripdata-jc/")
# clean_jc = client.map(clean_filenames, unzipped_jc_files, "data/citibike-tripdata-jc/")

client.gather(unzipped_nyc_files) # gather for many futures
